# **Bioinformatics with Jupyter Notebooks for WormBase:**
## **Data Retrieval 1 - Exploring WormBase FTP**
Welcome to the first jupyter notebook in the WormBase tutorial series. Over this series of tutorials, we will write code in Python that allows us to retrieve and perform simple analyses with data available on the WormBase sites.

This tutorial will deal with the data on the FTP site. (ftp://ftp.wormbase.org/pub/wormbase/)
We will both explore the site and extract data of interest. Let's get started!

We start by installing and loading the libraries that are required for this tutorial.

In [14]:
import wget
import gzip
import shutil
import ftplib
import os

We need to connect to the wormbase ftp site. Here, we navigate to the `pub/wormbase/` directory which is where the data is situated.
The cell below opens a ftp connection with the site and displays the files and sub-directories inside the `pub/wormbase/` directory.

In [12]:
ftp = ftplib.FTP('ftp.wormbase.org')
ftp.login()
ftp.cwd('pub/wormbase')
files = ftp.retrlines('LIST')
files

-rwxr-xr-x    1 1001     1001         2930 Aug 31  2020 README
-rwxr-xr-x    1 1001     1001         2622 Aug 14  2014 README~
drwxr-xr-x    3 1001     1001           20 Aug 24  2013 archive
drwxr-xr-x   18 1001     1001         4096 Apr 15 15:53 datasets-published
drwxr-xr-x   11 1001     1001         4096 Mar 31  2016 datasets-wormbase
lrwxrwxrwx    1 1001     1001           29 Aug 29  2013 nGASP -> datasets-published/nGASP_2005
drwxr-xr-x    3 1001     1001           22 May 24  2011 outgoing
drwxr-xr-x    7 1001     1001          103 Apr 22 10:44 parasite
drwxr-xr-x    4 1001     1001           34 May 17  2013 people
drwxr-xr-x  100 1001     1001        16384 Jun 11 19:15 releases
drwxr-xr-x    4 1001     1001         8192 Mar 02  2013 software
drwxr-xr-x   36 1001     1001         4096 Aug 08  2019 species


'226 Directory send OK.'

The README file contains information regarding the data contained in all the directories and subdirectories. Downloading it can lead you to required data easily.

In [13]:
with open('README', 'wb') as downloaded_file:
  ftp.retrbinary('RETR README', downloaded_file.write)

In [16]:
f = open('README').read()
print(f)

Site Contents
--------------------------------------

species/
   Core files and annotations for all species available
   at WormBase (or of possible interest to WormBase users)
   organized by species. Files previously available in genomes/
   can be found here.  File names, paths, and contents are 
   standardized and computable. Please see species/README for
   details.

      Look here for the most current and archival versions of:
        - genomic fasta sequence
        - genomic annotations in GFF2 or GFF3
        - assembly versions
        - commonly requests data sets by species
     
releases/
   Core files for each WormBase release organized by WS release ID.

      Check here if you are interested in downloading all the files
      that comprise the current WormBase release, or any other
      older releases.

datasets-published/
   Published datasets submitted to WormBase for distribution.

datasets-wormbase/
   WormBase-generated datasets and data dumps. Includes non-spe

The releases folder contains the core files for the various releases of WormBase and all subsidiary files. We can easily access the current version and download the required data. 

In [ ]:
!wget --cut-dirs 4 -r --no-parent ftp://ftp.wormbase.org/pub/wormbase/releases/current-development-release/
!mv ftp.wormbase.org current_release

The cell above downloads all the files associated with the current development release (which is around 40 GB in size). A lot of the data might not be relevant to the users requirement and that is why we can download just the required data from the release using the code in the following cells.

The following cells deal with accessing and downloading the required sequence, annotations, gff, and assembly data from the current development release.

To download assembly files, annotation files, etc., assign the organism name, the bioproject id, the wormbase version (WS280 for the latest version), the file type.
You can also naviagte into the different directories and check the available files to download.

Change the variables based on what you need. Uncomment the lines that list the files in a directory in case you need to see the available files in a directory before assigning your variables.

In [2]:
#print(*ftp.nlst('releases/current-development-release/species'), sep = "\n")
species = 'c_elegans'
#print(*ftp.nlst('releases/current-development-release/species/'+species), sep = "\n")
bioproject = 'PRJNA13758'
#print(*ftp.nlst('releases/current-development-release/species/'+species+'/'+bioproject), sep = "\n")
wormbase_id = 'WS280'
descriptor = 'annotations'
extension = 'gff3'

In [6]:
link = 'ftp://ftp.wormbase.org/pub/wormbase/releases/current-development-release/species/' + species
link += '/' + bioproject + '/' + species + '.' + bioproject + '.' + wormbase_id + '.' + descriptor 
link += '.' + extension + '.gz'
link

'ftp://ftp.wormbase.org/pub/wormbase/releases/current-development-release/species/c_elegans/PRJNA13758/c_elegans.PRJNA13758.WS280.annotations.gff3.gz'

Download the file from the link we generated above using the identifiers provided by you. And then you can unzip the .gz file to get the required file!

In [ ]:
wget.download(link)

In [7]:
downloaded_file = os.path.basename(link)



with gzip.open(downloaded_file + '.gz', 'rb') as f_in:
    with open(downloaded_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
# I would simply this ^^^^

'c_elegans.PRJNA13758.WS280.annotations.gff3.gz'

Here we can get the sequence files for a species - genomic, protein and transcript.

Again - change the variables based on what you need. Uncomment the lines that list the files in a directory in case you need to see the available files in a directory before assigning your variables.

In [9]:
#print(*ftp.nlst('species'), sep = "\n")
species = 'c_elegans'
data_type = 'sequence/protein'
#print(*ftp.nlst('species/'+species+'/'+data_type), sep = "\n")
bioproject = 'PRJNA13758'
wormbase_id = 'WS280'
# Descriptor can be - genomic.fa, genomic_masked.fa, genomic_softmasked.fa 
# OR protein.fa, wormpep_package.tar
# OR CDS_transcripts.fa, mRNA_transcripts.fa, cds_transcripts.fa, coding_transcripts.fa, ncrna_transcripts.fa, transposon_transcripts.fa, pseudogenic_transcripts.fa, ncRNA_transcripts.fa
descriptor = 'wormpep_package.tar'

In [10]:
if bioproject != '':
    bioproject = '.' + bioproject
link = 'ftp://ftp.wormbase.org/pub/wormbase/species/' + species + '/' + data_type + '/'
link += species + bioproject + '.' + wormbase_id + '.' + descriptor +'.gz'
link

'ftp://ftp.wormbase.org/pub/wormbase/species/c_elegans/sequence/protein/c_elegans.PRJNA275000.WS217.wormpep_package.tar.gz'

In [ ]:
wget.download(link)

Here we can get the gff files for a species.

Again - change the variables based on what you need. Uncomment the lines that list the files in a directory in case you need to see the available files in a directory before assigning your variables.

In [11]:
#print(*ftp.nlst('species'), sep = "\n")
species = 'c_elegans'
data_type = 'gff'
#print(*ftp.nlst('species/'+species+'/'+data_type), sep = "\n")
bioproject = ''
wormbase_id = 'WS280'
descriptor = 'annotations' #annotations or protein_annotation
file_extension='gff2' #gff2 or gff3

In [12]:
if bioproject != '':
    bioproject = '.' + bioproject
link = 'ftp://ftp.wormbase.org/pub/wormbase/species/' + species + '/' + data_type + '/'
link += species + bioproject + '.' + wormbase_id + '.' + descriptor + '.' + file_extension + '.gz'
link

'ftp://ftp.wormbase.org/pub/wormbase/species/c_elegans/gff/c_elegans.WS280.annotations.gff2.gz'

In [ ]:
wget.download(link)

Here we can get the assembly files for a species.

Again - change the variables based on what you need. Uncomment the lines that list the files in a directory in case you need to see the available files in a directory before assigning your variables.

In [ ]:
#print(*ftp.nlst('species'), sep = "\n")
species = 'c_elegans'
data_type = 'assemblies'
#print(*ftp.nlst('species/'+species+'/'+data_type), sep = "\n")
bioproject = 'PRJNA13758'
wormbase_id = 'current_development'
descriptor = 'assembly'

In [ ]:
if bioproject != '':
    bioproject = '.' + bioproject
link = 'ftp://ftp.wormbase.org/pub/wormbase/species/' + species + '/'
link += data_type+ '/' + species + bioproject + '.' + wormbase_id + '.' + descriptor + '.agp.gz'
link

In [ ]:
wget.download(link)

Here we can get the annotation files for a species.

Again - change the variables based on what you need. Uncomment the lines that list the files in a directory in case you need to see the available files in a directory before assigning your variables.

In [13]:
#Getting annotation files 
#print(*ftp.nlst('species'), sep = "\n")
species = 'c_elegans'
data_type = 'annotation'
print(*ftp.nlst('species/' + species + '/' + data_type), sep = "\n")

NameError: name 'ftp' is not defined

In [ ]:
type_of_annot = 'geneIDs'
print(*ftp.nlst('species/'+species+'/'+data_type+'/'+type_of_annot), sep = "\n")

In [ ]:
#Copy required file from above list to the variable below
download_file = 'species/c_elegans/annotation/geneIDs/c_elegans.PRJNA13758.current.geneIDs.txt.gz'
link = 'ftp://ftp.wormbase.org/pub/wormbase/'+download_file
link

In [ ]:
wget.download(link)

The following cells deal with accessing and downloading the required ontology data from the current development release.

Change the variables based on what you need. Uncomment the lines that list the files in a directory in case you need to see the available files in a directory before assigning your variables.

In [ ]:
#Types of data: anatomy_association, anatomy_ontology, development_association, development_ontology,
#disease_association, disease_ontology, gene_association, gene_ontology, phenotype_association, phenotype_ontology,
#rnai_phenotypes, rnai_phenotypes_quick.

data_type = 'gene_association'
species = 'c_elegans'
wormbase_id = 'WS280'

In [ ]:
if (species != ''):
    species = '.' + species

if (data_type[-8:]=='ontology'):
    link = 'ftp://ftp.wormbase.org/pub/wormbase/releases/current-development-release/ONTOLOGY/'+data_type+'.'+wormbase_id+'.obo'
elif (data_type[4:]=='rnai'):
    link = 'ftp://ftp.wormbase.org/pub/wormbase/releases/current-development-release/ONTOLOGY/'+data_type+'.'+wormbase_id+'.wb.c_elegans'
elif (data_type == 'disease_association'):
    link = 'ftp://ftp.wormbase.org/pub/wormbase/releases/current-development-release/ONTOLOGY/'+data_type+'.'+wormbase_id+'.daf.txt'+species
else :
    link = 'ftp://ftp.wormbase.org/pub/wormbase/releases/current-development-release/ONTOLOGY/'+data_type+'.'+wormbase_id+'.wb'+species

In [ ]:
wget.download(link)

This is the end of the first tutorial for WormBase data! This tutorial dealt with extracting WormBase data from the FTP site easily and programatically. 

In the next tutorial, we will use intermine to access the WormMine site and retrieve data from WormBase in another way.